In [31]:
!pip install python-docx openai psycopg2 numpy
!pip uninstall -y azure-search-documents
!pip uninstall -y azure-search-documents
!pip show azure-search-documents
!pip install --pre azure-search-documents==11.6.0b11
!pip install openai==0.28.1

Found existing installation: azure-search-documents 11.6.0b11
Uninstalling azure-search-documents-11.6.0b11:
  Successfully uninstalled azure-search-documents-11.6.0b11
  Using cached azure_search_documents-11.6.0b11-py3-none-any.whl (338 kB)
^C
ERROR: Operation cancelled by user


In [103]:
import os

print("Current working directory:", os.getcwd())
print("Files in 'data':", os.listdir("../data"))


Current working directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/amlicfg-ci3cf4d/code/Users/aoaitfs251786/azure-e2e-ai-fleet-maintenance-platform/notebooks
Files in 'data': ['.amlignore', '.amlignore.amltmp', 'Fleet maintenance best practices across industries.docx', 'Fleet vehicle maintenance A comprehensive guide.docx', 'Fleet_Maintenance_Tips.docx', 'raw_fleet_data.csv', 'test.txt', 'Top 10 tips on Fleet Preventive Maintenance.docx']


In [104]:
from docx import Document
from pathlib import Path

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

# Safer path handling
docx_files = [
    Path("../data") / "Fleet maintenance best practices across industries.docx",
    Path("../data") / "Top 10 tips on Fleet Preventive Maintenance.docx",
    Path("../data") / "Fleet vehicle maintenance A comprehensive guide.docx",
    Path("../data") / "Fleet_Maintenance_Tips.docx"
]

documents = {}

for file_path in docx_files:
    if not file_path.exists():
        print(f" File not found: {file_path}")
        continue
    file_name = file_path.name
    text = extract_text_from_docx(file_path)
    documents[file_name] = text

print(f" Successfully loaded {len(documents)} documents.")


 Successfully loaded 4 documents.


In [105]:
%%writefile .env
SUBSCRIPTIONID=791f5d76-9639-49f6-8681-d47dba4aa465
RESOURCEGROUP=rg-contosofleetguard-l3cf4d

AML_WORKSPACE_NAME=amlws-cfg-ws3cf4d
AML_COMPUTE_INSTANCE=amlicfg-ci3cf4d
AML_CLUSTER=amlccfg-aml-cluster

PG_HOST=azpostsql-cfg-psql3cf4d.postgres.database.azure.com
PG_DB=flexibleserverdb
PG_USER=citus
PG_PASSWORD=Fhtest208
PG_PORT=5432

AZURE_SEARCH_ENDPOINT=https://fleet-search-service3cf4d.search.windows.net
AZURE_SEARCH_KEY=R59zgdnZJqbIJB6NXBnIb1y7rTaEZtvJmmKXm2kGwuAzSeCXIEcG
AZURE_SEARCH_INDEX_NAME=fleet_index

AZURE_OPENAI_KEY=2t38UCg6oMyb4w4gATvxSkowr6JdDJfutieALZD2cXxSlVkmzC40JQQJ99BDACYeBjFXJ3w3AAABACOGecV5
AZURE_OPENAI_ENDPOINT= https://fleet34jhgh34.openai.azure.com/
AZURE_OPENAI_embedding_DEPLOYMENT=text-embedding-ada-002
AZURE_OPENAI_CHAT_DEPLOYMENT=gpt-35-turbo-16k


Overwriting .env


In [3]:
%%writefile .env

SUBSCRIPTIONID=791f5d76-9639-49f6-8681-d47dba4aa465
RESOURCEGROUP=rg-contosofleetguard-l3cf4d

AZURE_OPENAI_KEY=<Your Azure openAI keu>
AZURE_OPENAI_ENDPOINT= <Your Azure OpenAI Endpoint>
AZURE_OPENAI_DEPLOYMENT=<Your embedding deployment name>
AZURE_OPENAI_CHAT_DEPLOYMENT=<Your GPT chat model deployment name >

PG_HOST=<Your postgreSQL HosT>
PG_DB=<your db name>
PG_USER=<your db username>
PG_PASSWORD=<your db password>
PG_PORT=5432

AZURE_SEARCH_ENDPOINT=<Your Azure Serach service ennpoint>
AZURE_SEARCH_KEY=<Your Azure Serach service key>
AZURE_SEARCH_INDEX_NAME=fleet_index



Overwriting .env


In [106]:
#  Intelligent Fleet Maintenance - Prediction Upload and Search Index Notebook

import os
import pandas as pd
import time
from sqlalchemy import create_engine, text
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
import openai
import json

# ========================
# 1. Load Predictions from PostgreSQL
# ========================

load_dotenv(".env")

DB_CONNECTION_STRING = f"postgresql://{os.getenv('PG_USER')}:{os.getenv('PG_PASSWORD')}@{os.getenv('PG_HOST')}:{os.getenv('PG_PORT')}/{os.getenv('PG_DB')}"
engine = create_engine(DB_CONNECTION_STRING)

query = text("SELECT * FROM predictions;")
with engine.connect() as connection:
    result = connection.execute(query)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())

print(" Loaded predictions from PostgreSQL")

# ========================
# 2. Join with Maintenance Tips
# ========================

tip_query = text("SELECT * FROM maintenance_tips;")
with engine.connect() as connection:
    result = connection.execute(tip_query)
    tips_df = pd.DataFrame(result.fetchall(), columns=result.keys())

# Merge
merged_df = pd.merge(df, tips_df, on="maintenanceflag", how="left")

# Estimate cost from text field (if needed)
merged_df["estimated_cost"] = merged_df["cost_estimation"].fillna("Estimate Pending")

print(" Joined predictions with maintenance tips and cost estimates")

# ========================
# 3. Upload to Azure AI Search
# ========================

# Required format
merged_df["vehicleid"] = merged_df["vehicleid"].astype(str)

upload_df = merged_df[[
    "vehicleid", "enginehealth", "vehiclespeedsensor", "enginecoolanttemp", "enginerpm",
    "massairflowrate", "speedgps", "litresper100kminst", "co2ingperkminst",
    "triptimejourney", "maintenanceflag", "tip", "estimated_cost"
]]

records = upload_df.to_dict(orient="records")

# Azure AI Search config
search_service_name = "fleet-search-service3cf4d"
admin_key = "R59zgdnZJqbIJB6NXBnIb1y7rTaEZtvJmmKXm2kGwuAzSeCXIEcG"  #  REPLACE in prod!
index_name = "fleet-index"

search_client = SearchClient(
    endpoint=f"https://{search_service_name}.search.windows.net",
    index_name=index_name,
    credential=AzureKeyCredential(admin_key)
)

# Upload
batch_size = 1000
for i in range(0, len(records), batch_size):
    batch = records[i:i+batch_size]
    result = search_client.upload_documents(documents=batch)
    print(f" Uploaded {len(result)} records in batch {i // batch_size + 1}")

print(" All predictions uploaded to Azure AI Search index!")

# ========================
# 4. Generate Embeddings for Maintenance Tips and Save to PostgreSQL
# ========================

openai.api_type = "azure"
openai.api_base = "https://fleet34jhgh34.openai.azure.com/"
openai.api_key = "2t38UCg6oMyb4w4gATvxSkowr6JdDJfutieALZD2cXxSlVkmzC40JQQJ99BDACYeBjFXJ3w3AAABACOGecV5"
openai.api_version = "2023-05-15"
deployment_name = "text-embedding-ada-002"

def get_embedding(text):
    response = openai.Embedding.create(
        input=text,
        engine=deployment_name
    )
    return response['data'][0]['embedding']

with engine.begin() as conn:
    for idx, row in tips_df.iterrows():
        embedding = get_embedding(row['tip'])
        update_stmt = text("UPDATE maintenance_tips SET tip_embedding = :embedding WHERE id = :id")
        conn.execute(update_stmt, {"embedding": json.dumps(embedding), "id": row['id']})
        print(f" Updated tip_embedding for maintenanceflag {row['maintenanceflag']}")

print(" All tip embeddings updated in PostgreSQL")


 Loaded predictions from PostgreSQL
 Joined predictions with maintenance tips and cost estimates
 All predictions uploaded to Azure AI Search index!
 Updated tip_embedding for maintenanceflag 0
 Updated tip_embedding for maintenanceflag 1
 Updated tip_embedding for maintenanceflag 2
 Updated tip_embedding for maintenanceflag 3
 Updated tip_embedding for maintenanceflag 4
 All tip embeddings updated in PostgreSQL


In [107]:
from dotenv import load_dotenv
import os

# Explicitly load .env from current directory
load_dotenv(dotenv_path=".env")
print("AZURE_SEARCH_KEY:", os.getenv("AZURE_SEARCH_KEY"))


AZURE_SEARCH_KEY: JBdqudQ20M83rcTj3poEMtgGLk3UfnY7BKRNAGSj5KAzSeBrXu2E


In [108]:
import time
import openai
from dotenv import load_dotenv
from docx import Document
import os

# Load .env
load_dotenv()

# Azure OpenAI setup
openai.api_type = "azure"
openai.api_base = "https://fleet34jhgh34.openai.azure.com/"
openai.api_key = "2t38UCg6oMyb4w4gATvxSkowr6JdDJfutieALZD2cXxSlVkmzC40JQQJ99BDACYeBjFXJ3w3AAABACOGecV5"
openai.api_version = "2023-05-15"
deployment_name = "text-embedding-ada-002"

def get_embedding(text):
    while True:
        try:
            response = openai.Embedding.create(
                input=text,
                engine=deployment_name
            )
            return response['data'][0]['embedding']
        except openai.error.RateLimitError:
            print("Rate limit exceeded. Retrying in 60 seconds...")
            time.sleep(60)

# Assuming `documents` is a dict: {filename: content}
embeddings = {
    filename: get_embedding(content)
    for filename, content in documents.items()
}


In [109]:
import psycopg2
import numpy as np

# Connection details - update as per your PostgreSQL server
conn = psycopg2.connect(
    dbname=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    sslmode='require'  # required for Azure
)

cur = conn.cursor()

# Insert documents into PostgreSQL table
for filename, content in documents.items():
    embedding_vector = np.array(embeddings[filename]).tolist()
    
    cur.execute(
        "INSERT INTO maintenance_documents (filename, content, embeddings) VALUES (%s, %s, %s)",
        (filename, content, embedding_vector)
    )

conn.commit()
cur.close()
conn.close()
print("Documents and embeddings inserted successfully.")


Documents and embeddings inserted successfully.


In [110]:
import psycopg2
import numpy as np
import json  # Optional: to pretty-print

# Connect to the database
conn = psycopg2.connect(
    dbname=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    sslmode='require'  # required for Azure 
)

cur = conn.cursor()

# Fetch records from the table
cur.execute("SELECT filename, content, embeddings FROM maintenance_documents LIMIT 5")  # You can change the LIMIT

rows = cur.fetchall()

for row in rows:
    filename, content, embedding = row
    print(f"Filename: {filename}")
    print(f"Content: {content[:100]}...")  # Print a snippet of content
    print(f"Embedding Vector Length: {len(embedding)}")
    print(f"First 5 Embedding Values: {embedding[:5]}\n")  # Show first few values



cur.close()
conn.close()


Filename: Fleet maintenance best practices across industries.docx
Content: Fleet maintenance best practices across industries.
Fleet maintenance helps keep vehicles in good co...
Embedding Vector Length: 19370
First 5 Embedding Values: [0.01

Filename: Top 10 tips on Fleet Preventive Maintenance.docx
Content: Top 10 tips on Fleet Preventive Maintenance
Discover the top 10 tips on preventive maintenance and l...
Embedding Vector Length: 19411
First 5 Embedding Values: [0.01

Filename: Fleet vehicle maintenance A comprehensive guide.docx
Content: Fleet vehicle maintenance: A comprehensive guide
Fleet vehicle maintenance is essential for keeping ...
Embedding Vector Length: 19327
First 5 Embedding Values: [0.01

Filename: Fleet_Maintenance_Tips.docx
Content: Fleet Maintenance Tips & Cost Estimation Guide
This document provides a detailed guide on fleet main...
Embedding Vector Length: 19363
First 5 Embedding Values: [-0.0

Filename: Fleet maintenance best practices across industries.docx


In [111]:
query_text = "how to reduce fleet repair costs"
query_embedding = get_embedding(query_text)


In [112]:
import psycopg2
import numpy as np

# Assume query_embedding is a list or numpy array
query_embedding = np.array(query_embedding).tolist()

# Connect to the database
conn = psycopg2.connect(
    dbname=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    sslmode='require'
)
cur = conn.cursor()

# SQL to get only the embeddings of top 5 most similar records
sql = """
    SELECT embeddings
    FROM maintenance_documents
    ORDER BY embeddings <=> %s::vector
    LIMIT 5;
"""

cur.execute(sql, (query_embedding,))
results = cur.fetchall()

# Each result is a tuple with one element: the embedding vector
for (embedding,) in results:
    print(embedding)  # Just prints the full embedding vector

cur.close()
conn.close()


[0.01469292,0.0028073974,0.005231073,-0.028729906,-0.00884199,-0.0026450537,0.0010863906,-0.0051392424,0.0008084386,-0.0059821177,-0.015034006,0.026919529,-0.015834246,0.013643426,-0.00041979773,0.0018513736,0.037073385,-0.013085882,0.017644623,-0.013217069,-0.029280892,-0.009917721,-0.026473494,-0.015230786,-0.0014069784,-0.0052901073,0.028546246,-0.021501515,-0.022419821,-0.008914142,0.016267162,-0.007457969,0.0029910589,0.0008281166,-0.008697684,0.0020645522,0.026079934,-0.031904627,-0.010776995,-0.01567682,0.03646993,0.016726315,-0.003843773,-0.002551583,-0.00023900594,0.021081718,0.01380085,0.010409672,-0.009825891,0.007471088,-0.0011634629,0.005381938,-0.014364953,-0.011505081,-0.011524759,0.0019054881,-0.009294584,0.036889724,-0.017001808,-0.023390604,-0.019796086,0.018510455,-0.028861094,0.0006731522,0.0010560537,-0.0003515396,0.00228429,0.017106757,-0.019336933,-0.008953499,0.025581423,0.03484321,0.012856306,0.008376276,0.0054376926,-0.03437094,-0.0024499132,0.0030468132,0.009

In [113]:
query_text = "how to reduce fleet repair costs"
query_embedding = get_embedding(query_text)


## export

In [114]:
import psycopg2
import json

# PostgreSQL connection details
conn = psycopg2.connect(
    dbname=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    sslmode='require'  # required for Azure 
)

cur = conn.cursor()

# Fetch filename, content, and embeddings from the table
cur.execute("""
    SELECT filename, content, embeddings
    FROM maintenance_documents;
""")

records = cur.fetchall()
cur.close()
conn.close()

# Convert to list of documents for Azure AI Search
documents = []
for i, (filename, content, embedding) in enumerate(records):
    documents.append({
        "@search.action": "upload",
        "id": str(i),
        "filename": filename,
        "content": content,
        "embedding": embedding  # Ensure this is a list of floats
    })

# Optional: save as JSON file for inspection or upload
with open("fleet_docs_with_embeddings.json", "w") as f:
    json.dump(documents, f)

print(" Export complete. Ready to push to Azure AI Search.")


 Export complete. Ready to push to Azure AI Search.


In [115]:
import json
import psycopg2
import os
from dotenv import load_dotenv
import ast  # Safe way to evaluate string to list

# Load environment variables
load_dotenv()

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=os.getenv("PG_HOST"),
    dbname=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    port=os.getenv("PG_PORT")
)

cur = conn.cursor()

# Query the documents
cur.execute("SELECT id, filename, content, embeddings FROM maintenance_documents")

# Build document list
documents = []
for row in cur.fetchall():
    id_, filename, content, embedding = row

    # Safely convert string to list of floats
    try:
        embedding_list = ast.literal_eval(embedding) if isinstance(embedding, str) else embedding
        embedding_floats = [float(x) for x in embedding_list]
    except Exception as e:
        print(f"Failed to convert embedding for id {id_}: {e}")
        continue

    doc = {
        "id": str(id_),
        "filename": filename,
        "content": content,
        "embedding": embedding_floats
    }
    documents.append(doc)

# Write to JSON file
with open("maintenance_documents.json", "w", encoding="utf-8") as f:
    json.dump(documents, f, ensure_ascii=False, indent=2)

# Cleanup
cur.close()
conn.close()

print(" maintenance_documents.json created successfully.")


 maintenance_documents.json created successfully.


### Updated endpont and key values with your search service values  

In [116]:
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential

endpoint = "https://fleet-search-service3cf4d.search.windows.net"
key = "R59zgdnZJqbIJB6NXBnIb1y7rTaEZtvJmmKXm2kGwuAzSeCXIEcG"

try:
    client = SearchIndexClient(endpoint=endpoint, credential=AzureKeyCredential(key))
    indexes = client.list_indexes()
    print(" Connected to Azure Search. Index list:")
    for index in indexes:
        print("-", index.name)
except Exception as e:
    print(" Failed to connect:", e)


 Connected to Azure Search. Index list:
- fleet_index


In [117]:
import json
import os
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

# Load environment variables from .env (if you're using one)
load_dotenv()

# Config
endpoint = "https://fleet-search-service3cf4d.search.windows.net"
admin_key = "R59zgdnZJqbIJB6NXBnIb1y7rTaEZtvJmmKXm2kGwuAzSeCXIEcG"
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
json_file_path = "maintenance_documents.json"

# Check if environment variables are loaded correctly
if not endpoint or not admin_key:
    raise ValueError("Environment variables for endpoint or API key are not set correctly.")

# Upload documents
def upload_documents():
    credential = AzureKeyCredential(admin_key)
    search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            documents = json.load(f)
        print(f" Uploading {len(documents)} documents to index '{index_name}'...")

        result = search_client.upload_documents(documents=documents)

        succeeded = [r for r in result if r.succeeded]
        failed = [r for r in result if not r.succeeded]

        print(f" Upload finished. Succeeded: {len(succeeded)} | Failed: {len(failed)}")
        if failed:
            for f in failed:
                print("Fail", f.error_message)

    except Exception as e:
        print(f" Upload failed: {e}")

# Run upload
upload_documents()


 Uploading 16 documents to index 'fleet_index'...
 Upload finished. Succeeded: 16 | Failed: 0


AZURE_OPENAI_KEY=
AZURE_OPENAI_ENDPOINT=
AZURE_OPENAI_DEPLOYMENT=

# Azure AI Search settings
search_service_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
search_api_key = "R59zgdnZJqbIJB6NXBnIb1y7rTaEZtvJmmKXm2kGwuAzSeCXIEcG"

# Azure OpenAI settings
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  # Example: https://your-openai-resource.openai.azure.com/
openai.api_version = "2023-05-15"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
embedding_deployment = os.getenv("AZURE_OPENAI_embedding_DEPLOYMENT")  # Your deployed embedding model name

In [118]:
import os
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path=".env")

# Azure AI Search settings
search_service_endpoint = "https://fleet-search-service3cf4d.search.windows.net"
index_name = "fleet_index"
search_api_key = "R59zgdnZJqbIJB6NXBnIb1y7rTaEZtvJmmKXm2kGwuAzSeCXIEcG"

# Azure OpenAI settings
openai.api_type = "azure"
openai.api_base = "https://fleet34jhgh34.openai.azure.com/"  # Example: https://your-openai-resource.openai.azure.com/
openai.api_version = "2023-05-15"
openai.api_key = "2t38UCg6oMyb4w4gATvxSkowr6JdDJfutieALZD2cXxSlVkmzC40JQQJ99BDACYeBjFXJ3w3AAABACOGecV5"
embedding_deployment = "text-embedding-ada-002" # Your deployed embedding model name

# Initialize Azure Search client
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

# Get embedding from Azure OpenAI
def get_embedding(text: str):
    response = openai.Embedding.create(
        input=[text],
        deployment_id=embedding_deployment  
    )
    return response['data'][0]['embedding']

# Search using vector - return only the best answer
def search_index(query_text, top_k=1):
    try:
        vector = get_embedding(query_text)

        vector_query = VectorizedQuery(
            vector=vector,
            fields="embedding"
        )

        results = search_client.search(
            search_text=" ",  # required
            vector_queries=[vector_query]
        )

        best_result = next(results, None)
        if best_result:
            print(f"\nAnswer for: \"{query_text}\"\n")
            print(best_result.get("content", "")[:500], "...\n")
        else:
            print(" No relevant results found.")

    except Exception as e:
        print(" Error during search:", e)

        print(f"\n Top {top_k} results for: \"{query_text}\"\n")
        for result in results:
            print(" Filename:", result.get("filename", "N/A"))
            print(" Snippet:", result.get("content", "")[:300], "...\n")

    except Exception as e:
        print(" Error during search:", e)

# Example usage
search_index("How do I fix overheating issues?")



Answer for: "How do I fix overheating issues?"

Top 10 tips on Fleet Preventive Maintenance
Discover the top 10 tips on preventive maintenance and learn how regular upkeep like oil changes, brake pad replacement, battery inspection, and transmission fluid checks can save you money by reducing the cost of ownership while increasing productivity and reducing operational costs. Simply Fleet, A fleet management app for small business helps achieve this goal.
To ensure that your fleet vehicle runs smoothly, following a preventive maintenance check ...



update with your chat completion model name -engine

In [119]:
# Search using vector + generate answer using gpt model
def search_index(query_text, top_k=1):
    try:
        # Step 1: Get query embedding
        vector = get_embedding(query_text)

        vector_query = VectorizedQuery(
            vector=vector,
            fields="embedding"
        )

        # Step 2: Search top-k results
        results = search_client.search(
            search_text=" ",
            vector_queries=[vector_query]
        )

        best_result = next(results, None)

        if best_result:
            document_content = best_result.get("content", "")

            # Step 3: Generate answer using gpt model
            prompt = f"""
You are a helpful assistant for fleet vehicle maintenance.

Use the following context from a document to answer the question.

Context:
\"\"\"
{document_content}
\"\"\"

Question: {query_text}
Answer:"""

            completion = openai.ChatCompletion.create(
                engine="gpt-35-turbo-16k",  #  your chat completion deployment name
                messages=[
                    {"role": "system", "content": "You are a helpful assistant for fleet maintenance."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                max_tokens=300
            )

            answer = completion['choices'][0]['message']['content']
            print(f"\n Answer for: \"{query_text}\"\n")
            print(answer)

        else:
            print(" No relevant results found.")

    except Exception as e:
        print("Error during search:", e)


# 3. Azure AI Search setup
search_service_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
search_api_key = os.getenv("AZURE_SEARCH_KEY")

# 4. Azure OpenAI setup
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  # e.g., https://your-resource-name.openai.azure.com/
openai.api_version = "2023-05-15"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
embedding_deployment = os.getenv("AZURE_OPENAI_embedding_DEPLOYMENT")  # Your embedding model name
chat_model_deployment = "os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT") " # Your chat model deployment name

In [120]:
# 1. Import necessary packages
import os
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

# 2. Load environment variables
load_dotenv(dotenv_path=".env")

# 3. Azure AI Search setup
search_service_endpoint = "https://fleet-search-service3cf4d.search.windows.net"
index_name = "fleet_index"
search_api_key = "R59zgdnZJqbIJB6NXBnIb1y7rTaEZtvJmmKXm2kGwuAzSeCXIEcG"

# 4. Azure OpenAI setup
openai.api_type = "azure"
openai.api_base = "https://fleet34jhgh34.openai.azure.com/"  # e.g., https://your-resource-name.openai.azure.com/
openai.api_version = "2023-05-15"
openai.api_key = "2t38UCg6oMyb4w4gATvxSkowr6JdDJfutieALZD2cXxSlVkmzC40JQQJ99BDACYeBjFXJ3w3AAABACOGecV5"
embedding_deployment = "text-embedding-ada-002"  # Your embedding model name
chat_model_deployment = "gpt-35-turbo-16k" # Your chat model deployment name

# 5. Initialize search client
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

# 6. Function to get embeddings from Azure OpenAI
def get_embedding(text: str):
    response = openai.Embedding.create(
        input=[text],
        deployment_id=embedding_deployment
    )
    return response['data'][0]['embedding']

# 7. Vector search + GPT response
def search_index(query_text, top_k=1):
    try:
        # Get query embedding
        vector = get_embedding(query_text)

        vector_query = VectorizedQuery(
            vector=vector,
            fields="embedding"
        )

        # Vector search
        results = search_client.search(
            search_text=" ",  # required
            vector_queries=[vector_query]
        )

        best_result = next(results, None)

        if best_result:
            document_content = best_result.get("content", "")

            # Prompt for GPT to answer using the document content
            prompt = f"""
You are a helpful assistant for fleet vehicle maintenance.

Use the following context from a document to answer the question.

Context:
\"\"\"
{document_content}
\"\"\"

Question: {query_text}
Answer:"""

            completion = openai.ChatCompletion.create(
                deployment_id=chat_model_deployment,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant for fleet maintenance."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                max_tokens=300
            )

            answer = completion['choices'][0]['message']['content']
            print(f"\n Answer for: \"{query_text}\"\n")
            print(answer)

        else:
            print("No relevant results found.")

    except Exception as e:
        print(" Error during search:", e)

# 8. Run the actual query
search_index("How do I fix overheating issues?")



 Answer for: "How do I fix overheating issues?"

To fix overheating issues in a fleet vehicle, you can take the following steps:

1. Check the coolant level: Ensure that the coolant level is sufficient and topped up if necessary. Coolant helps maintain optimal operating temperatures and prevents overheating.

2. Inspect for coolant leaks: Look for any signs of coolant leaks, such as puddles under the vehicle or visible coolant dripping. If a leak is found, it should be repaired to prevent further overheating.

3. Check the radiator: Make sure the radiator is not clogged or blocked. Clean any debris or dirt that may be obstructing the airflow through the radiator.

4. Inspect the radiator fan: Ensure that the radiator fan is functioning properly. If the fan is not working, it may need to be repaired or replaced.

5. Flush the cooling system: Over time, the cooling system can accumulate debris and contaminants that can hinder its effectiveness. Flushing the cooling system can help remov

Push fleet_cleaned data to  aml-index - upload your DataFrame to Azure AI Search aml-index:

In [124]:
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# Load env
load_dotenv()

# Setup database connection
DB_CONNECTION_STRING = f"postgresql://{os.getenv('PG_USER')}:{os.getenv('PG_PASSWORD')}@{os.getenv('PG_HOST')}:{os.getenv('PG_PORT')}/{os.getenv('PG_DB')}"
engine = create_engine(DB_CONNECTION_STRING)

#  Vehicle input data (customize values)
vehicle_input = {
    "enginehealth": 0.78,
    "vehiclespeedsensor": 55,
    "enginecoolanttemp": 100,
    "enginerpm": 2800,
    "massairflowrate": 90.1,
    "speedgps": 70,
    "litresper100kminst": 14.2,
    "co2ingperkminst": 330.0,
    "triptimejourney": 200
}

#  Simulate model prediction (you'll replace with real model later)
predicted_flag = 2  # e.g., model.predict(...) gives 2

#  Fetch matching tip and cost from PostgreSQL
query = text("SELECT tip, cost_estimation FROM maintenance_tips WHERE maintenanceflag = :flag")
with engine.connect() as conn:
    tip_row = conn.execute(query, {"flag": predicted_flag}).fetchone()

#  Final result
result = {
    **vehicle_input,
    "maintenanceflag": predicted_flag,
    "maintenance_tip": tip_row[0],
    "estimated_cost": tip_row[1]
}

# Display
pd.DataFrame([result])


,enginehealth,vehiclespeedsensor,enginecoolanttemp,enginerpm,massairflowrate,speedgps,litresper100kminst,co2ingperkminst,triptimejourney,maintenanceflag,maintenance_tip,estimated_cost
0,0.78,55,100,2800,90.1,70,14.2,330.0,200,2,Moderate maintenance required. Check engine he...,$150 - $300
